In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
}

s = requests.Session()
s.headers.update(headers)


#s.get('https://www.alpsonline.org/reservation/calendar?hut_id=27')
#r = s.get('https://www.alpsonline.org/reservation/selectDate?date=10.07.2024')
#json_list = []
#for i, value in r.json().items():
#    json_list.append(value[0])
#df = pd.DataFrame(json_list)

In [6]:
hutid = [27, 515]
date = '10.07.2027'

#loop over all huts, create a dataframe for each hut
#initialize dataframes
df_list = []

for hut in hutid:
    r = s.get(f'https://www.alpsonline.org/reservation/calendar?hut_id={hut}')
    r = s.get(f'https://www.alpsonline.org/reservation/selectDate?date={date}')
    json_list = []
    for i, value in r.json().items():
        json_list.append(value[0])
    df = pd.DataFrame(json_list)
    df['hutid'] = hut
    df_list.append(df)


#reduce dataframes to relevant columns, add hutid and calculate occupation
df_red_list = []
for df in df_list:
    df_red = df[['hutid', 'reservationDate', 'freeRoom']].copy()
    df_red['occupation'] = (100 - df_red['freeRoom'].copy() / df['totalRoom'].copy() * 100)
    df_red['Service'] = df['bedCategoryType'].copy()
    df_red_list.append(df_red)

#concatenate dataframes
df_con = pd.concat(df_red_list)
print(df_con)


    hutid reservationDate  freeRoom  occupation     Service
0      27      10.07.2027        53         0.0        ROOM
1      27      11.07.2027        53         0.0        ROOM
2      27      12.07.2027        53         0.0        ROOM
3      27      13.07.2027        53         0.0        ROOM
4      27      14.07.2027        53         0.0        ROOM
5      27      15.07.2027        53         0.0        ROOM
6      27      16.07.2027        53         0.0        ROOM
7      27      17.07.2027        53         0.0        ROOM
8      27      18.07.2027        53         0.0        ROOM
9      27      19.07.2027        53         0.0        ROOM
10     27      20.07.2027        53         0.0        ROOM
11     27      21.07.2027        53         0.0        ROOM
12     27      22.07.2027        53         0.0        ROOM
13     27      23.07.2027        53         0.0        ROOM
0     515      10.07.2027        10         0.0  UNSERVICED
1     515      11.07.2027        10     